# Shipment Tracking 
A Pipeline & Dashboard to track shipments sent through multiple vendors <hr>

Key Project Elements:
- Deepnote Dashboard elements for intaking data from *end user*
- API interaction and (probably) web scraping to fetch shipment status from vendor site 
- `pydantic` for data validation of shipment information
- read and write to `.csv` for data permanence
    - afterward may include `Great Expectations` to track data quality

## Setup

### Resources
- [Python API Tutorial](https://www.dataquest.io/blog/python-api-tutorial/)
- [`BeautifulSoup4` Documentation](https://pypi.org/project/beautifulsoup4/)
    - [Tutorial](https://realpython.com/beautiful-soup-web-scraper-python/) on getting started with `bs4`
- [How to](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.append.html) append rows to a `pandas` dataframe
- [`Pydantic` Documentation](https://pydantic-docs.helpmanual.io/)
- [`Great Expectations` tutorial](https://towardsdatascience.com/monitoring-data-quality-in-a-data-lake-using-great-expectations-and-allure-built-serverless-47fa1791af6a)

### Import statements

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs4
import json
import requests as re
import datetime as dt
import pydantic
from typing import Optional
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
print("All packages imported successfully")

All packages imported successfully


### Initialize Data
import original data from `.csv`

In [ ]:
x = dt.date(2022, 8, 1)
print(x)

2022-08-01


In [ ]:
date_cols:list = ['Shipment Date', 'ETA', 'Delivery Date']
new_date_cols:list = ['shipped_date', 'eta_date', 'delivered_date']

def autoconvert_datetime(value):
    formats = ['%m/%d/%Y','%m/%d/%y', '%m-%d-%y', '%d-%B', '%d %B']  # formats to try
    result_format = '%Y-%m-%d'  # output format
    exception_ct = 0
    for dt_format in formats:
        try:
            dt_obj = dt.datetime.strptime(value, dt_format)
            return dt_obj.strftime(result_format)
        except Exception as e:  # throws exception when format doesn't match
            exception_ct += 1
            pass
    #print(f"Total number of exceptions: {exception_ct}")
    return value  # let it be if it doesn't match


shipments = (pd.read_csv('./Data/Freight_Tracking.csv')
                .drop(columns=['Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16'])
                .fillna("Not Entered")
                .replace("ASAP","Not Entered")
            )
for c in date_cols:
    shipments[c] = shipments[c].apply(autoconvert_datetime)
shipments.isnull().sum()

SO#                       0
PO#                       0
Customer                  0
Destination / Zip Code    0
Shipment Date             0
Carrier                   0
PRO# Wass                 0
PRO#                      0
Broker Quote              0
Invoice amount            0
ETA                       0
Delivery Date             0
Transit Time              0
Status                    0
dtype: int64

In [ ]:
shipments.sample(5)

In [ ]:
d = shipments.ETA[1]
dt.datetime.strptime(d, '%m/%d/%y')
autoconvert_datetime(d)

ValueError: time data '2021-06-17' does not match format '%m/%d/%y'

In [ ]:
date_cols:list = ['Shipment Date', 'ETA', 'Delivery Date']
new_date_cols:list = ['shipped_date', 'eta_date', 'delivered_date']

def autoconvert_datetime(value):
    formats = ['%m/%d/%Y','%m/%d/%y', '%m-%d-%y', '%d-%B', '%d %B']  # formats to try
    result_format = '%d-%m-%Y'  # output format
    exception_ct = 0
    for dt_format in formats:
        try:
            dt_obj = dt.datetime.strptime(value, dt_format)
            return dt_obj.strftime(result_format)
        except Exception as e:  # throws exception when format doesn't match
            exception_ct += 1
            pass
    #print(f"Total number of exceptions: {exception_ct}")
    return value  # let it be if it doesn't match

shipments['ETA'] = shipments['ETA'].apply(autoconvert_datetime)

Total number of exceptions: 5
Total number of exceptions: 5
Total number of exceptions: 5
Total number of exceptions: 5
Total number of exceptions: 5
Total number of exceptions: 5


In [ ]:
shipments

In [ ]:
shipments.isnull().sum()

SO#                       0
PO#                       0
Customer                  0
Destination / Zip Code    0
Shipment Date             0
Carrier                   0
PRO# Wass                 0
PRO#                      0
Broker Quote              0
Invoice amount            0
ETA                       0
Delivery Date             0
Transit Time              0
Status                    0
shipped_date              0
eta_date                  0
delivered_date            0
dtype: int64

In [ ]:
x = set(shipments.Customer)
x

In [ ]:
shipments["Shipment Date"] = pd.to_datetime(shipments["Shipment Date"])

In [ ]:
shipments.dtypes

SO#                               object
PO#                               object
Customer                          object
Destination / Zip Code            object
Shipment Date             datetime64[ns]
Carrier                           object
PRO# Wass                         object
PRO#                              object
Broker Quote                      object
Invoice amount                   float64
ETA                               object
Delivery Date                     object
Transit Time                      object
Status                            object
Unnamed: 14                      float64
Unnamed: 15                       object
Unnamed: 16                       object
dtype: object

In [ ]:
shipments.sample(5)

### Test Cleaning up Columns

In [ ]:
customers = set(shipments["Customer"].str.strip())
customers

## Main Code

In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs4
import json
import requests as re
import datetime as dt
import pydantic
from typing import Optional
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
print("All packages imported successfully")

All packages imported successfully


### General Data Cleaning & Model
Apply general cleaning procedures to `shipments` dataframe to:
- remove whitespace
- format dates
    - handle inconcsistent date formats
    - format to `datetime` object
- handle missing data

<hr>

The code above was getting messy trying to initialize everything with a dataframe. I think instead we can:
1. create a `pydantic` model for an arbitrary `Shipment` instance of a shipment, then 
2. iterate through the `.csv` file and make each row a new instance of `Shipment()`
    a. add each instance of `Shipment()` to a new `DataFrame`

In doing this, we'll see if we can make a `pydantic` model that is able to handle all the above cleaning tasks through its parsing operations

Define adaptable date conversion function

In [ ]:
def autoconvert_datetime(value, formats:list=['%m/%d/%Y','%m/%d/%y', '%m-%d-%y', '%d-%B', '%d %B'], additional_formats_to_try:list=[]):
    """An adaptable function to try taking in date strings for conversion to `datetime` objects.
        Can take user-defined date formats either replacing defaults in `formats` 
        or by including extra formats in `additional_formats_to_try`
    """
    formats.extend(additional_formats_to_try)    # all formats to try

    result_format = '%Y-%m-%d'  # output format
    
    exception_ct = 0 # set a counter for exceptions
    
    # iterate through given formats to try converting to datetime
    for dt_format in formats:
        try:
            dt_obj = dt.datetime.strptime(value, dt_format)
            return str(dt_obj.strftime(result_format))
        except Exception as e:  # throws exception when format doesn't match
            exception_ct += 1
            pass
    #print(f"Total number of exceptions: {exception_ct}")
    return value  # let it be if it doesn't match

Define generalized function to import data

In [ ]:
# generalized function to import data
def import_shipping_data(data_csv:str) -> pd.DataFrame:
    date_cols:list = ['Shipment Date', 'ETA', 'Delivery Date']
    valid_cols:list = ['SO#', 'PO#', 'Customer', 'Destination / Zip Code', 'Shipment Date', 
                        'Carrier', 'PRO# Wass', 'PRO#', 'Broker Quote', 'Invoice amount', 
                        'ETA','Delivery Date', 'Transit Time', 'Status'
                        ]

    shipments_df = (pd.read_csv(data_csv, usecols=valid_cols)
                    .fillna("Not Entered")
                    .replace("ASAP","Not Entered")
                )
    for c in date_cols:
        shipments_df[c] = shipments_df[c].apply(autoconvert_datetime)
    
    return shipments_df

Import Data

In [ ]:
shipments = import_shipping_data('./Data/Freight_Tracking.csv')

In [ ]:
shipments.columns

Index(['SO#', 'PO#', 'Customer', 'Destination / Zip Code', 'Shipment Date',
       'Carrier', 'PRO# Wass', 'PRO#', 'Broker Quote', 'Invoice amount', 'ETA',
       'Delivery Date', 'Transit Time', 'Status'],
      dtype='object')

### Set up Data Structure with `Pydantic` Models

Determine valid PO# lengths

In [ ]:
l = []
for i in range(len(shipments)):
    l.append(len(shipments["PO#"][i]))
x = set(l)
print(x)
for j in x:
    print(f"Valid PO# length: {j}")

{5, 6, 7, 8, 9, 10, 11, 12, 13, 14}
Valid PO# length: 5
Valid PO# length: 6
Valid PO# length: 7
Valid PO# length: 8
Valid PO# length: 9
Valid PO# length: 10
Valid PO# length: 11
Valid PO# length: 12
Valid PO# length: 13
Valid PO# length: 14


#### Set up valid carriers and `Carrier` class

In [ ]:
class Carrier(pydantic.BaseModel):
    carrier_name: str
    tracking_url: pydantic.AnyHttpUrl

    class Config:
        anystr_strip_whitespace = True
        extra = pydantic.Extra.ignore

In [ ]:
carrier_data = [{"carrier_name":"Bunzl", "tracking_url":"https://www.xyz.com"}, {"carrier_name":"Road Runner", "tracking_url":"https://www.xyz.com"}]
carriers:list[Carrier] = [Carrier(**item) for item in carrier_data]
#print(c1)

In [ ]:
c0 = carriers[0]
c0

Carrier(carrier_name='Bunzl', tracking_url=AnyHttpUrl('https://www.xyz.com', scheme='https', host='www.xyz.com', tld='com', host_type='domain'))

In [ ]:
class X(pydantic.BaseSettings):
    a: str = pydantic.Field(...,env='shell')

In [ ]:
import os
print(os.environ)

environ({'SHELL': '/bin/bash', 'KUBERNETES_SERVICE_PORT_HTTPS': '443', 'DEEPNOTE_GITHUB_TOKEN_SERVICE_URL': 'https://deepnote.com/api/github-apps/git-token', 'JUPYTER_CONFIG_DIR': '/deepnote-config/jupyter', 'KUBERNETES_SERVICE_PORT': '443', 'INTEGRATION_CREDENTIALS_SERVICE_PORT': 'tcp://10.100.12.67:80', 'HOSTNAME': 'p-23421d57-1fd0-41ae-ba7e-62fc693f5e69', 'PYTHON_VERSION': '3.10', 'LANGUAGE': 'en_US.UTF-8', 'GIT_SSH_COMMAND': 'ssh -i /work/.deepnote/gitkey -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no', 'INTEGRATION_CREDENTIALS_SERVICE_PORT_80_TCP_ADDR': '10.100.12.67', 'INTEGRATION_CREDENTIALS_SERVICE_PORT_80_TCP': 'tcp://10.100.12.67:80', 'PWD': '/', 'JUPYTER_TOKEN': '2492d054-84de-46ce-8d41-0ee51b4b2af2', 'PYTHON_SETUPTOOLS_VERSION': '58.1.0', 'INTEGRATION_CREDENTIALS_SERVICE_SERVICE_PORT': '80', 'PLOTLY_RENDERER': 'colab', 'IPYTHONDIR': '/deepnote-config/ipython', 'HOME': '/root', 'LANG': 'en_US.UTF-8', 'KUBERNETES_PORT_443_TCP': 'tcp://10.100.0.1:443', 'INTEGRATIO

In [ ]:
p = X()
p.dict()

{'a': '/bin/bash'}

#### Set up `Shipment` class

In [ ]:
from datetime import date, datetime, time, timedelta
class Shipment(pydantic.BaseModel):
    SO_num: str      # internal shipment order nmber
    PO_num: str      # Product order number (tracking number generated by vendor)
    customer: str    # customer name
    carrier: str     # carrier (shipping vendor) name
    destination: str # where the shipment is going
    shipped_date: date # date the package was shipped
    eta: Optional[date] = None # date the package is expected to arrive
    delivered_date: Optional[date] = None
    transit_time: Optional[int] = None
    status: str = "In-Transit"
    #'PRO# Wass', 'PRO#', 'Broker Quote', 'Invoice amount'
    pro_num_wass: Optional[str] = None
    pro_num: Optional[str] = None
    broker_quote: Optional[float] = None
    invoice_amt: Optional[float] = None

    class Config:
        anystr_strip_whitespace = True
        extra = pydantic.Extra.allow

    @pydantic.validator("PO_num") # may refine this once I understand PO# formats required for each carrier
    @classmethod
    def valid_PO_num(cls, value):
        chars = [c for c in value if c in "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ-"]
        invalid_chars = [i for i in value if i not in "0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ-"]
        
        if value in [None, "Not Entered", np.nan]:
            raise MissingPOError(value=value, message=message)
        
        elif len(chars) not in [5, 6, 7, 8, 9, 10, 11, 12, 13, 14]:
            raise InvalidPOError(value=value, message="PO number given is not valid")        

        return value 

    @pydantic.validator("carrier") # may refine this once I understand PO# formats required for each carrier
    @classmethod
    def validate_carrier(cls, value):
        valid_carriers = [carriers[i].carrier_name for i in range(len(carriers))]
        msg = "shipment carrier/vendor is not in the pre-approved list of carriers/vendors...please check spelling or add the new carrier with its tracking site url. Vendors/carreirs can be added with the add_new_carrier(name, url) function"

        v = value.replace(" ","").lower()
        
        if v not in valid_carriers:
            raise InvalidCarrierError(value=value, message=msg)
        
        return value

    
    
    #def calculate_transit_time(self, self.shipped_date,self.)

#### Create custom error classes
<hr>
PO# error classes:

In [ ]:
class InvalidPOError(Exception):
    """custom error that is raised whe PO# contains invalid characters or is not one of the pre-approved formats or lengths"""
    def __init__(self,value:str, message:str) -> None:
        self.value = value
        self.message = message
        super().__init__(message)

class MissingPOError(InvalidPOError):
    def __init__(self,value:str, message:str) -> None:
        self.value = value
        self.message = "PO# was not entered! Please go back and enter PO# for this shipment"
        super().__init__(message)


Carrier error classes:

In [ ]:
class InvalidCarrierError(Exception):
    """custom error raised when the shipment carrier/vendor is not in the pre-approved list
        of carriers/vendors. Vendors/carreirs can be added with the add_new_carrier(name, url) function"""
    def __init__(self,value:str, message:str) -> None:
        self.value = value
        self.message = message
        super().__init__(message)

test with a fake shipment

In [ ]:
s1 = Shipment(
    SO_num = "121334",
    PO_num = "123456",
    customer = "   Perkins Reyes ",
    carrier = "Bunzl   ",
    destination = "MN, 55433",
    shipped_date = '2022-01-09',
    delivered_date = '2022-01-14'
)

In [ ]:
s1.dict()

{'SO_num': '121334',
 'PO_num': '123456',
 'customer': 'Perkins Reyes',
 'carrier': 'Bunzl',
 'destination': 'MN, 55433',
 'shipped_date': datetime.date(2022, 1, 9),
 'eta': None,
 'delivered_date': datetime.date(2022, 1, 14),
 'transit_time': None,
 'status': 'In-Transit',
 'pro_num_wass': None,
 'pro_num': None,
 'broker_quote': None,
 'invoice_amt': None}

Further clean up and format columns `shipments` dataframe to create global variables for each column that will be used as input/dataclass options later

## Dashboard (End User Interface)

### Global Variables

In [ ]:
#carriers_from_data = list(set(shipments['Carrier']))
carriers_from_data = (shipments.Carrier
                            .str.replace(" ","")
                            .str.lower()
                            .unique()
                        )
print(carriers_from_data)
print(len(carriers_from_data))

['conway' 'estes' 'reddaway' 'abf' 'roadrunner' 'r&l' 'glova-link' 'yrc'
 'centralfreight' 'olddominion' 'upsfreight' 'saia' 'tforce' 'oakharbor'
 'centraltransport' 'bestyet' 'hercules' 'frontline' 'dhe' 'xpo']
20


In [ ]:
customers_from_data = (shipments.Customer
                            .str.replace(" ","")
                            .str.lower()
                            .unique()
                        )
print(customers_from_data)
print(len(customers_from_data))

['bunzl' 'gmri' 'wasswerstrom' 'notentered' 'cdc' 'imperialdade'
 'wasserstrom' 'ross' 'buccaneer' 'kinexo-mclane' 'lineage-fortworth'
 'spiritmart' 'mallory' 'eddon' 'magidglove' 'imperialdale' 'trimark'
 'lineage-wilmington' 'lineage' 'dotit']
20


### Set up Interface Elements

In [ ]:
po_input = ''

In [ ]:
customer_select = 'bunzl'

In [ ]:
carrier_select = 'carrier_name=\'Bunzl\' tracking_url=AnyHttpUrl(\'https://www.xyz.com\', scheme=\'https\', host=\'www.xyz.com\', tld=\'com\', host_type=\'domain\')'

In [ ]:

from datetime import datetime as _deepnote_datetime
eta_input = _deepnote_datetime.strptime('2022-08-06T07:00:00.000Z', "%Y-%m-%dT%H:%M:%S.%fZ")


In [ ]:
add_valid_carrier = 'USPS'

In [ ]:
new_carrier_url = ''

In [ ]:
carriers.append(add_valid_carrier)
print(carriers)

['Road Runner ', 'Roadrunner ', 'Central Transport', 'Conway ', 'T Force', 'Frontline', 'ABF', 'DHE', 'Saia', 'Roadrunner', 'Best Yet', 'XPO', 'SAIA', 'YRC', 'Ups Freight ', 'ESTES', 'Oak Harbor ', 'Conway', 'Reddaway', 'Estes', 'Tforce', 'Old Dominion', 'Hercules', 'R&L', 'Glova - Link', 'R&L ', 'Central Freight', 'USPS']


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=23421d57-1fd0-41ae-ba7e-62fc693f5e69' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>